In [27]:
import numpy as np

class User:
    def __init__(self, name):
        self.name = name
        self.sent_message = []  # List containing messages sent encoded
        self.received_message = []  # List containing messages received decoded

    def add_sent_message(self, message):
        self.sent_message.append(message)

    def add_received_message(self, decoded_message):
        self.received_message.append(decoded_message)

class Message:
    def initializer(self, from_user, to_user, metadata, message_body):
        self.from_user = from_user  # User sending the message
        self.to_user = to_user  # User receiving the message
        self.metadata = metadata  # Metadata for the message
        self.message_body = message_body  # The content of the message (string)

    def lossy_compress(self, lossiness_factor):
        print(f"Original Message: {self.message_body}")  # print the original message

        # transform the message into its frequency domain using fft
        message_array = np.array([ord(char) for char in self.message_body])  # convert characters to ascii codes
        transformed = np.fft.fft(message_array)  # perform fast fourier transform

        # keep only the most significant frequencies based on the lossiness factor
        threshold = int(len(transformed) * lossiness_factor)  # calculate how many frequencies to retain
        compressed_transform = np.zeros_like(transformed)  # create a zeroed array for compression
        compressed_transform[:threshold] = transformed[:threshold]  # copy the most significant frequencies

        # convert the modified frequency data back to the time domain
        lossy_message = np.fft.ifft(compressed_transform)  # perform inverse fft
        # reconstruct the message, keeping only printable characters
        lossy_message = "".join(
            chr(int(np.real(c))) for c in lossy_message if 32 <= np.real(c) < 127
        )

        print(f"Lossy Compressed Message (Lossiness {lossiness_factor * 100}%): {lossy_message}")

        # update metadata with the original message length
        self.metadata = f"original_length={len(self.message_body)}"
        # replace the original message with the lossy compressed version
        self.message_body = lossy_message

    def send(self, lossiness_factor=0.8):  # default lossiness factor is 80%
        # compress the message using lossy compression before sending
        self.lossy_compress(lossiness_factor)
        # store the compressed message in the sender's sent_message list
        self.from_user.add_sent_message(self.message_body)
        # store the compressed message in the receiver's received_message list
        self.to_user.add_received_message(self.message_body)
        
# Test cases
userA = User("Alice")
userB = User("Bob")

# Test with 80% lossiness
print("\nTest 1\n80% Lossiness")
messageA = Message(userA, userB, "", "This is a detailed message.")
messageA.send(lossiness_factor=0.8)  # Retain 80% of the frequencies
print("Results:")
print("Sent Messages (User A):", userA.sent_message[-1:])
print("Received Messages (User B):", userB.received_message[-1:])

# Test with 100% lossiness (no data loss)
print("\nTest 2\n100% Lossiness")
messageB = Message(userA, userB, "", "This is a detailed message.")
messageB.send(lossiness_factor=1.0)  # Retain 100% of the frequencies
print("Results:")
print("Sent Messages (User A):", userA.sent_message[-1:])
print("Received Messages (User B):", userB.received_message[-1:])



Test 1
80% Lossiness
Original Message: This is a detailed message.
Lossy Compressed Message (Lossiness 80.0%): \a^s*np$s2fYk^c_^l-o[gjY`h;
Results:
Sent Messages (User A): ['\\a^s*np$s2fYk^c_^l-o[gjY`h;']
Received Messages (User B): ['\\a^s*np$s2fYk^c_^l-o[gjY`h;']

Test 2
100% Lossiness
Original Message: This is a detailed message.
Lossy Compressed Message (Lossiness 100.0%): This is a detailed message.
Results:
Sent Messages (User A): ['This is a detailed message.']
Received Messages (User B): ['This is a detailed message.']
